In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

In [18]:
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [19]:
application_df = application_df.drop(columns=['EIN','NAME',"SPECIAL_CONSIDERATIONS","STATUS","ASK_AMT"])
application_df.head(20)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,1
5,T3,Independent,C1200,Preservation,Trust,0,1
6,T3,Independent,C1000,Preservation,Trust,100000-499999,1
7,T3,Independent,C2000,Preservation,Trust,10M-50M,1
8,T7,Independent,C1000,ProductDev,Trust,1-9999,1
9,T5,CompanySponsored,C3000,ProductDev,Association,0,0


In [20]:
cutoff_value = 200
data = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = data[data < cutoff_value].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [21]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
class_cutoff_value = 1000
class_data = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = class_data[class_data < class_cutoff_value].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [22]:
# Convert categorical data to numeric with `pd.get_dummies`
convert_data = pd.get_dummies(application_df[['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION','USE_CASE','INCOME_AMT','ORGANIZATION']])
#convert_data.head()

application_df = pd.concat([application_df,convert_data],axis = 1)

application_df = application_df.drop(columns=['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION','USE_CASE','INCOME_AMT','ORGANIZATION'])

application_df.head()

,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1


In [23]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"]
X = application_df.drop(columns = "IS_SUCCESSFUL")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y ,random_state=42)

In [24]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [25]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.#  YOUR CODE GOES HERE
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=130,activation="relu",input_dim=39))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=50, activation="sigmoid"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

#Compile
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Check the structure of the model
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 130)               5200      
                                                                 
 dense_7 (Dense)             (None, 50)                6550      
                                                                 
 dense_8 (Dense)             (None, 1)                 51        
                                                                 
Total params: 11801 (46.10 KB)
Trainable params: 11801 (46.10 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                                  patience=5,
                                                  mode='min',
                                                  restore_best_weights=True)

In [27]:
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100,callbacks=[early_stopping])

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5704 - accuracy: 0.7185
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5559 - accuracy: 0.7277
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5534 - accuracy: 0.7297
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5516 - accuracy: 0.7300
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5500 - accuracy: 0.7297
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5493 - accuracy: 0.7314
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5483 - accuracy: 0.7317
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5477 - accuracy: 0.7302
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5478 - accuracy: 0.7326
Epoch 10/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5476 - accura

In [13]:
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5662 - accuracy: 0.7292 - 1s/epoch - 5ms/step
Loss: 0.5662410855293274, Accuracy: 0.7292128205299377


In [14]:
# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity_Optimization.h5")

/Users/cameron-wadecarson/anaconda3/envs/dev/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
